## Include file path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/Shareddrives/AME508_Project/lib')
sys.path.append('/content/drive/Shareddrives/AME508_Project/data')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## import package

In [ ]:
import numpy as np
import tensorflow as tf
import scipy
import scipy.io

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

from tensorflow import keras
%matplotlib inline 
import matplotlib
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models
from keras.regularizers import l2

Found GPU at: /device:GPU:0


## load traning data

In [ ]:
import h5py

with h5py.File('/content/drive/Shareddrives/AME508_Project/data/train_val_data_two/train_dataset_two.mat', 'r') as file:
    train_data = list(file['data_train'])

with h5py.File('/content/drive/Shareddrives/AME508_Project/data/train_val_data_two/val_dataset_two.mat', 'r') as file:
    val_data = list(file['data_val'])


In [ ]:
train_y = train_data[4]

train_x1 = np.zeros((len(train_y),2))
train_x1[:,0] = train_data[2]
train_x1[:,1] = train_data[3]
# train_x1 = np.reshape(train_x1,(len(train_y),1))

train_x2 = np.zeros((len(train_x1),2))
train_x2[:,0] = train_data[0]
train_x2[:,1] = train_data[1]

val_y = val_data[4]
val_x1 = np.zeros((len(val_y),2))
val_x1[:,0] = val_data[2]
val_x1[:,1] = val_data[3]
val_x2 = np.zeros((len(val_x1),2))
val_x2[:,0] = val_data[0]
val_x2[:,1] = val_data[1]


## define Neural network

In [ ]:
class MyModel(tf.keras.Model):

  def __init__(self,input_dim_1,input_dim_2,dot_dim,width,depth,reg_param=1e-6):
    super(MyModel, self).__init__()
    Reg_Func = l2
    self.depth=depth
    self.layers1=[]
    self.layers2=[]

    for i  in range(self.depth-1):
      bias_ini=keras.initializers.RandomUniform(minval=-1, maxval=1, seed=None)
      kernel_ini = keras.initializers.RandomUniform(minval=-1, maxval=1, seed=None)
      self.layers1.append(tf.keras.layers.Dense(width, activation=tf.nn.tanh, kernel_initializer=kernel_ini, bias_initializer=bias_ini,kernel_regularizer=Reg_Func(reg_param)))
      self.layers2.append(tf.keras.layers.Dense(width, activation=tf.nn.tanh, kernel_initializer=kernel_ini, bias_initializer=bias_ini,kernel_regularizer=Reg_Func(reg_param)))
    self.layers1.append(tf.keras.layers.Dense(dot_dim, activation=tf.nn.tanh, kernel_initializer=kernel_ini, bias_initializer=bias_ini,kernel_regularizer=Reg_Func(reg_param)))
    self.layers2.append(tf.keras.layers.Dense(dot_dim, activation=tf.nn.tanh, kernel_initializer=kernel_ini, bias_initializer=bias_ini,kernel_regularizer=Reg_Func(reg_param)))


  def call(self, inputs_1, inputs_2):
    for i in range(self.depth):
      layer1=self.layers1[i]
      layer2=self.layers2[i]
      inputs_1=layer1(inputs_1)
      inputs_2=layer2(inputs_2)
    return tf.reduce_sum( tf.multiply( inputs_1, inputs_2 ), 1 )

In [ ]:
def train():

  tf.keras.backend.clear_session()
  input_dim_1=2
  input_dim_2=2
  dot_dim=30
  width=15
  depth=15
  reg_param=1e-6
  learning_rate=1e-3
  model=MyModel(input_dim_1=input_dim_1,input_dim_2=input_dim_2,dot_dim=dot_dim,width=width,depth=depth,reg_param=reg_param)
  
  optimizer   = keras.optimizers.Adam(learning_rate=learning_rate) 
  max_epoch   = 1000000
  batch_size  = 64

  los=[]
  val_los = []
  for epoch in range(1,max_epoch+1):
      # print(epoch)
      train_batch_index=np.random.choice(len(train_y),batch_size)
      val_batch_index=np.random.choice(len(val_y),batch_size)
      with tf.GradientTape() as tape1:
        out=model(train_x1[train_batch_index,:],train_x2[train_batch_index,:])
        val_out = model(val_x1[val_batch_index,:],val_x2[val_batch_index,:])
        # model.summary()
        # print(tf.shape(out))
        loss=tf.reduce_mean(tf.square(out-train_y[train_batch_index]))
        val_loss = tf.reduce_mean(tf.square(val_out-val_y[val_batch_index]))
      los.append(loss.numpy())
      val_los.append(val_loss.numpy())

      grads = tape1.gradient(loss, model.trainable_variables)
      
      optimizer.apply_gradients(zip(grads, model.trainable_variables)) # zip used to create an iterator over the tuples
      # print(loss)
      if epoch % 100 == 0 or epoch==max_epoch:
          print(f"Epoch: {epoch}, loss: {loss:.2e}, val_loss: {val_loss:.2e}")
      if epoch % 5000 == 0  or epoch==max_epoch:
          model.save_weights('/content/drive/Shareddrives/AME508_Project/code/trained_model/model/model_{}'.format(epoch))
      # if epoch % 100 == 0:
      #     print(grads)
  return  model,los#,loss_int,loss_bc

## training

In [ ]:
model,loss=train()

## load Neutral Network

In [ ]:
input_dim_1=2
input_dim_2=2
dot_dim=30
width=15
depth=15
reg_param=1e-6
learning_rate=1e-3

model=MyModel(input_dim_1=input_dim_1,input_dim_2=input_dim_2,dot_dim=dot_dim,width=width,depth=depth,reg_param=reg_param)

file_name = '/content/drive/Shareddrives/AME508_Project/code/trained_model/model/model_1000000'
model.load_weights(file_name)


## optimizer

In [ ]:
import random
import scipy.io
import h5py
import numpy as np

file=scipy.io.loadmat('/content/drive/Shareddrives/AME508_Project/data/observation/14.mat')
known_data = file['u']

with h5py.File('/content/drive/Shareddrives/AME508_Project/data/observation/grid.mat', 'r') as file:
    x = list(file['x'])
    t = list(file['t'])

x_t = np.zeros((1000*1000,2))
x = np.reshape(x,(1000*1000,))
t = np.reshape(t,(1000*1000,))
x_t[:,0] = x;
x_t[:,1] = t;


# a = np.zeros((2,))
# a[0] = random.uniform(1e-4, 1e-1)
# a[1] = random.uniform(1,10)
a = tf.Variable([[0.05,5]], name='a', trainable=True, dtype=tf.float32)
optimizer   = keras.optimizers.Adam(1e-2) 
max_epoch = 500
for epoch in range(1,max_epoch+1):

  with tf.GradientTape() as tape:

    predict_output = model.call(a,x_t)

    # predict_output = output.numpy()

    loss = tf.reduce_mean(tf.square(predict_output-known_data))

  trainable_variables = [a]
  grads = tape.gradient(loss, trainable_variables)

  optimizer.apply_gradients(zip(grads, trainable_variables))
  if epoch % 10 == 0 or epoch==max_epoch:
    print(f"Epoch: {epoch}, a: {a.numpy()}, loss: {loss:.2e}")

## tfco (tensorflow constrained optimization)
(experiment)

In [ ]:
#######################
# import tensorflow_constrained_optimization as tfco

# class optProblem(tfco.ConstrainedMinimizationProblem):
#   def __init__(self, loss_fn,var,known_data,x_t):
#     self._loss_fn = loss_fn
#     self._var = var
#     self._known_data = known_data;
#     self._x_t = x_t;
#   @property
#   def num_constraints(self):
#     return 4
  
#   def objective(self):
#     return loss_fn(self._x_t,self._known_data)

#   def constraints(self):
#     a1, a2 = self._var
#     l1 = 1e-4 - a1
#     r1 = a1-1e-1
#     l2 = 1-a2
#     r2 = a2 - 10
#     constraints = tf.stack([l1,r1,l2,r2])
#     return constraints

# def loss_fn(x_t,known_data):

#   predict_output = model.call(tf.reshape(tf.stack([a1,a2]),[1,2]),x_t)
#   loss = tf.reduce_mean(tf.square(predict_output-known_data))
#   return loss

# # a = tf.Variable([[0.1,1]], name='a', trainable=True, dtype=tf.float32)
# a1 = tf.Variable(0.05, dtype=tf.float32, name='a1')
# a2 = tf.Variable(5.0, dtype=tf.float32, name='a2')

# file=scipy.io.loadmat('/Users/huangchenchen/Dropbox/ame508_testing/measured_data_0.0017_3.3333.mat')
# known_data = file['u']
# with h5py.File('/Users/huangchenchen/Dropbox/ame508_testing/grid.mat', 'r') as file:
#     x = list(file['x'])
#     t = list(file['t'])
# x_t = np.zeros((1000*1000,2))
# x = np.reshape(x,(1000*1000,))
# t = np.reshape(t,(1000*1000,))
# x_t[:,0] = x;
# x_t[:,1] = t;

# problem = optProblem(loss_fn, [a1,a2], known_data,x_t)

# optimizer = tfco.LagrangianOptimizerV2(
#     optimizer=tf.optimizers.Adagrad(learning_rate=1e-2),
#     num_constraints=problem.num_constraints
# )
# var_list = [a1,a2] + list(problem.trainable_variables) + optimizer.trainable_variables()

# for i in range(10000):
#     optimizer.minimize(problem, var_list=var_list)
#     if i % 10 == 0:
#         # print(f'step = {i}')
#         # print(f'loss = {loss_fn(x_t,known_data)}')
#         print(f"step = {i}, a1: {a1.numpy()}, a2: {a2.numpy()}, loss = {loss_fn(x_t,known_data):.2e}")

###########################3
